# PostgresDB Manager Usage Examples

## Introduction
This notebook demonstrates how to use the `PostgresDB` manager class to create, manage, and interact with a PostgreSQL database in a Docker container. We'll use Python and SQL cell magic to showcase the package functionality.

## Setup

### Install Required Packages

In [7]:
!pip install ipython-sql psycopg2-binary docker_db


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import Dependencies

In [8]:
import uuid
from pathlib import Path
from docker_db.postgres_db import PostgresConfig, PostgresDB

# For SQL cell magic
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Creating a PostgreSQL Database Instance

Let's create a temporary directory for our database files:

In [9]:
import tempfile
import os

# Create a temporary directory
temp_dir = Path(tempfile.mkdtemp())
print(f"Created temporary directory: {temp_dir}")

Created temporary directory: C:\Users\acisse\AppData\Local\Temp\tmp501158zs


Now, let's set up the PostgresDB configuration:

In [10]:
# Generate a unique container name
container_name = f"demo-postgres-{uuid.uuid4().hex[:8]}"

# Create a configuration for our database
config = PostgresConfig(
    user="demouser",
    password="demopass",
    database="demodb",
    project_name="demo",
    workdir=temp_dir,
    container_name=container_name,
    retries=20,
    delay=3,
)

# Initialize the database manager
db_manager = PostgresDB(config)

## Start the Database

We'll now create and start the database:

In [11]:
# Create and start the database
db_manager.create_db()
print(f"Database started successfully in container '{container_name}'")
print(f"Connection details:")
print(f"  Host: {config.host}")
print(f"  Port: {config.port}")
print(f"  User: {config.user}")
print(f"  Database: {config.database}")

Building image demo-postgres:dev...


RuntimeError: Failed to build image: {'message': 'Cannot locate specified Dockerfile: docker\\Dockerfile.pgdb'}

## Connect and Run SQL Queries

Now that our database is running, let's connect to it using SQL cell magic:

In [ ]:
# Define the connection string for SQL magic
conn_string = f"postgresql://{config.user}:{config.password}@{config.host}:{config.port}/{config.database}"
%sql $conn_string

### Creating a Table

In [ ]:
%%sql
CREATE TABLE demo_users (
    id SERIAL PRIMARY KEY,
    username VARCHAR(50) UNIQUE NOT NULL,
    email VARCHAR(100) UNIQUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

### Insert Data

In [ ]:
%%sql
INSERT INTO demo_users (username, email) VALUES
    ('alice', 'alice@example.com'),
    ('bob', 'bob@example.com'),
    ('charlie', 'charlie@example.com');

### Query Data

In [ ]:
%%sql
SELECT * FROM demo_users;

### Run More Complex Queries

In [ ]:
%%sql
-- Create another table for demonstration
CREATE TABLE demo_posts (
    id SERIAL PRIMARY KEY,
    user_id INTEGER REFERENCES demo_users(id),
    title VARCHAR(100) NOT NULL,
    content TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Insert some posts
INSERT INTO demo_posts (user_id, title, content) VALUES
    (1, 'Alice First Post', 'Hello world from Alice!'),
    (1, 'Alice Second Post', 'Another post from Alice'),
    (2, 'Bob Introduction', 'Hi, this is Bob!'),
    (3, 'Charlie Notes', 'Some notes from Charlie');
    
-- Query with JOIN
SELECT u.username, p.title, p.content
FROM demo_users u
JOIN demo_posts p ON u.id = p.user_id
ORDER BY u.username, p.created_at;

## Using Regular Python to Access the Database

You can also interact with the database using Python code and psycopg2:

In [ ]:
import psycopg2
from psycopg2.extras import RealDictCursor

# Connect to the database
conn = psycopg2.connect(
    host=config.host,
    port=config.port,
    database=config.database,
    user=config.user,
    password=config.password,
    cursor_factory=RealDictCursor
)

# Create a cursor
cur = conn.cursor()

# Execute a query
cur.execute("SELECT COUNT(*) as post_count FROM demo_posts")
result = cur.fetchone()
print(f"Total number of posts: {result['post_count']}")

# Group by query
cur.execute("""
    SELECT u.username, COUNT(p.id) as post_count 
    FROM demo_users u
    LEFT JOIN demo_posts p ON u.id = p.user_id
    GROUP BY u.username
    ORDER BY post_count DESC
""")

print("\nPost count by user:")
for row in cur.fetchall():
    print(f"  {row['username']}: {row['post_count']} posts")

# Close the connection
cur.close()
conn.close()

## Clean Up

When you're done with the database, you can delete it:

In [ ]:
# Delete the database container
db_manager.delete_db()
print(f"Database container '{container_name}' deleted")

# Clean up the temporary directory
import shutil
shutil.rmtree(temp_dir)
print(f"Temporary directory '{temp_dir}' removed")

## Complete Example with Error Handling

Here's a more complete example with proper error handling:

In [ ]:
from contextlib import contextmanager

@contextmanager
def temporary_postgres_db(db_name="tempdb", user="tempuser", password="temppass"):
    """Create a temporary PostgreSQL database and clean up after use."""
    # Create temporary directory
    temp_dir = Path(tempfile.mkdtemp())
    container_name = f"temp-postgres-{uuid.uuid4().hex[:8]}"
    
    # Configure database
    config = PostgresConfig(
        user=user,
        password=password,
        database=db_name,
        project_name="temp",
        workdir=temp_dir,
        container_name=container_name,
        retries=15,
        delay=2
    )
    
    # Create manager
    manager = PostgresDB(config)
    
    try:
        # Start the database
        manager.create_db()
        print(f"Temporary database started at {config.host}:{config.port}")
        
        # Yield the manager and connection info to the caller
        yield manager, config
        
    finally:
        # Clean up resources
        try:
            manager.delete_db()
            print(f"Database container '{container_name}' deleted")
        except Exception as e:
            print(f"Warning: Error deleting database container: {e}")
            
        try:
            shutil.rmtree(temp_dir)
            print(f"Temporary directory '{temp_dir}' removed")
        except Exception as e:
            print(f"Warning: Error removing temporary directory: {e}")


# Usage example:
if __name__ == "__main__":
    with temporary_postgres_db() as (db_manager, config):
        # Connect to database
        conn = psycopg2.connect(
            host=config.host,
            port=config.port,
            database=config.database,
            user=config.user,
            password=config.password
        )
        
        # Create a test table and insert data
        with conn.cursor() as cur:
            cur.execute("CREATE TABLE test (id SERIAL PRIMARY KEY, name TEXT)")
            cur.execute("INSERT INTO test (name) VALUES ('test data')")
            conn.commit()
            
            # Query the data
            cur.execute("SELECT * FROM test")
            print(cur.fetchall())
            
        # Close connection
        conn.close()

## Conclusion

This notebook demonstrated how to:

1. Configure and create a PostgreSQL database with `PostgresDB`
2. Connect to the database using SQL cell magic
3. Execute SQL queries on the database
4. Clean up the database when finished

The `PostgresDB` manager provides a convenient way to spin up PostgreSQL instances in Docker containers for development, testing, or demonstration purposes.